# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [ ]:
# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [11]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(f"Exception while connection to local cluster / establishing session - {e}")


#### Create Keyspace

In [12]:
# creating keyspace udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)    


#### Set Keyspace

In [13]:
# setting keyspace udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)    


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [16]:


## Query 1: gives artist_name, song_title and song_length from the music app history for given session_id and song_items parameters 
## Example: session_id = 338, and session_items = 4
## Primary key uses session_id and session_items. Table is partitioned by partition key session_id 
## and ordered by clustering column session_items withing the primary key.

query = """ create table if not exists user_session_history
            (session_id             int,
             session_items          int,
             artist_name            text,
             song_title             text,
             song_length            float,
             primary key ((session_id), session_items))"""

try:
    session.execute(query)
except Exception as e:
    print(e)



                    

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## building the insert query
        query = """insert into user_session_history (session_id, session_items, artist_name, song_title, song_length) 
                   values (%s, %s, %s, %s, %s)"""
    
        # assigning the data to the query from the csv line
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
## This is the Query 1 select to check the work done above is working
query = "select artist_name, song_title, song_length from user_session_history where session_id = 338 and session_items=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [20]:

## Query 2: Gives artist_name, song_title (sorted by session_items) and user_name (lastname, firstname) for a given user_id and session_id
## Example: userid = 10, sessionid = 182
## Primary key uses userid, session_id and session_items. 
## Table is partitioned by partition key userid, session_id 
## and ordered by clustering column session_items as outlined in the requirement

query = """create table if not exists user_session_item_history 
           (user_id int,
            session_id int,
            session_items int,
            artist_name   text,
            user_name     text,
            song_title    text,
            primary key ((user_id,session_id),session_items)
            
           )"""

try:
    session.execute(query)
except Exception as e:
    print(e)


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_item_history (user_id, session_id, session_items, artist_name, user_name, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[0], line[4] + ", " + line[1], line[9]))

query = "select artist_name, song_title, user_name from user_session_item_history where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)                  

In [25]:
## Query 3: Gives user_name for a given song_title
## Example: song_title ='All Hands Against His Own'
## Table is partitioned by partition key song_title 
## and ordered by clustering column user_id within the partition. 

query = """create table if not exists user_session_song_history 
           (song_title text,
            user_id int,
            user_name text,
            primary key ((song_title),user_id)
            
           )"""

try:
    session.execute(query)
except Exception as e:
    print(e)


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_song_history (song_title, user_id, user_name)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[4]+", "+line[1]) )

query = "select user_name from user_session_song_history where song_title ='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)      

                    

Row(song_title='All Hands Against His Own', user_name='Lynch, Jacqueline')
Row(song_title='All Hands Against His Own', user_name='Levine, Tegan')
Row(song_title='All Hands Against His Own', user_name='Johnson, Sara')


Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_name='Cruz, Sylvie')
Row(artist_name='Three Drives', song_title='Greece 2000', user_name='Cruz, Sylvie')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_name='Cruz, Sylvie')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_name='Cruz, Sylvie')


### Drop the tables before closing out the sessions

In [26]:
# dropping tables to facilitate repeated running of the program

query = "drop table user_session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session_item_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_session_song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()